In [1]:
import pandas as pd
import sqlite3
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np

In [17]:
querry = "SELECT rs.id, rs.user_id, corpus, lien FROM app_veille_ressource AS rs INNER JOIN app_veille_type_ressource AS tr ON rs.ressource_id = tr.id WHERE tr.name ='Article' and user_id = 1"
con = sqlite3.connect("/home/merouane/Gitlab/Ecole_IA/projet-5-groupe-3/Veille_IA/db.sqlite3")
df = pd.read_sql_query(sql=querry, con=con)

In [18]:
df

,id,user_id,corpus,lien
0,6,1,"L'apprentissage automatique[1],[2] (en anglais...",https://fr.wikipedia.org/wiki/Apprentissage_au...
1,7,1,Study of algorithms that improve automatically...,https://en.wikipedia.org/wiki/Machine_learning
2,8,1,"Un réseau de neurones artificiels[1],[2], ou r...",https://fr.wikipedia.org/wiki/R%C3%A9seau_de_n...
3,9,1,: On désigne ainsi toute méthode permettant de...,https://dataanalyticspost.com/Lexique/reductio...
4,10,1,Il existe différents types d’analyse de donnée...,https://datascientest.com/data-analysis-tout-s...
5,11,1,L'exploration de données est un processus de r...,https://docs.microsoft.com/fr-fr/analysis-serv...
6,12,1,Tout ce que vous devez savoir sur le Data Mini...,https://www.oracle.com/fr/database/data-mining...
7,13,1,"L’exploration de données[notes 1], connue auss...",https://fr.wikipedia.org/wiki/Exploration_de_d...
8,14,1,Le partitionnement de données (ou data cluster...,https://fr.wikipedia.org/wiki/Partitionnement_...
9,15,1,TensorFlow est un outil open source d'apprenti...,https://fr.wikipedia.org/wiki/TensorFlow


In [4]:
import spacy
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer


nlp = spacy.load('fr_core_news_md')
def clean_text(text):
    with open("/home/merouane/Gitlab/Ecole_IA/projet-5-groupe-3/Veille_IA/utils/stop_words_french.txt") as file:
        stop_words = file.read()
    stop_words = stop_words.split()
    
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.match('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    token_text=" ".join(filtered_tokens)
    #Lemmatizations
    doc = nlp(token_text)
    cleaned_text = [token.lemma_ for token in doc if str(token) not in stop_words and len(str(token))>3]
    cleaned_text = " ".join(cleaned_text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text

In [5]:
df["cleaned_data"] = df["corpus"].map(clean_text)

In [6]:
    vocabulary = set()

    for doc in df.cleaned_data:
        vocabulary.update(doc.split(' '))

    vocabulary = list(vocabulary)

In [7]:
with open("/home/merouane/Gitlab/Ecole_IA/projet-5-groupe-3/Veille_IA/utils/stop_words_french.txt") as file:
    stop_words = file.read()
stop_words = stop_words.split()

In [8]:
vectorizer = TfidfVectorizer(stop_words=stop_words, vocabulary=vocabulary)
X = vectorizer.fit_transform(df["cleaned_data"])

/home/merouane/.local/share/virtualenvs/projet-5-groupe-3-BHgmIQkK/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'daren', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'itse', 'll', 'mayn', 'mightn', 'mustn', 'myse', 'needn', 'oughtn', 'quelqu', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


In [9]:
def gen_vector_T(tokens):
    Q = np.zeros((len(vocabulary)))   
    x= vectorizer.transform(tokens)
    #print(tokens[0].split(','))
    for token in tokens:
        try:
            ind = vocabulary.index(token)
            Q[ind] = x[0, vectorizer.vocabulary_[token]]
        except:
            pass
    return Q

In [10]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [11]:
def cosine_similarity_T(k, query):
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] = clean_text(tokens[0])
    d_cosines = []
    query_vector = gen_vector_T(q_df['q_clean'])
    for d in X.A:

        d_cosines.append(cosine_sim(query_vector, d))

                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'id'] = df['id'][index]
        a.loc[i,'id_user'] = df['user_id'][index]
        a.loc[i,'Subject'] = df['lien'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [12]:
df_result = cosine_similarity_T(6,'machine learning')

In [13]:
id_ressource = [int(ressource) for ressource in df_result["id"]]

In [14]:
df_result

,id,id_user,Subject,Score
0,35.0,2.0,https://en.wikipedia.org/wiki/Machine_learning,0.271993
1,33.0,2.0,https://datascientest.com/machine-learning-tou...,0.171476
2,38.0,2.0,https://en.wikipedia.org/wiki/Natural_language...,0.153524
3,34.0,2.0,https://fr.wikipedia.org/wiki/Apprentissage_au...,0.081116
4,37.0,2.0,https://experiences.microsoft.fr/articles/inte...,0.055001
5,36.0,2.0,https://datascientest.com/introduction-au-nlp-...,0.025051


In [16]:
id_ressource

[35, 33, 38, 34, 37, 36]